# 1. Background

- Tornadoes occur at a high frequency in the United States compared with other natural hazards such as earthquakes and tsunamis but have a substantially smaller footprint. Even a single high-intensity tornado can result in high casualty rates and catastrophic economic losses and social consequences, particularly for small to medium communities.
- The city of Joplin, Missouri, USA, was hit by an EF-5 tornado on May 22, 2011.  The National Institute of Standards and Technology (NIST) conducted a technical investigation of this devastating event which can be found at: https://nvlpubs.nist.gov/nistpubs/NCSTAR/NIST.NCSTAR.3.pdf .  The Center for Risk-Based Community Resilience Planning  simulated this event for buildings and the electrical power network of Joplin in IN-CORE.  This Juypter Notebook provides an example of how to use IN-CORE.
- The initial damage prediction utilized the tornado path, tornado fragility curves representative of a 19- archetype building dataset, and EPN datasets.  Generic tornado paths are also available in IN-CORE, or a user defined tornado path is possible. 
- The functionality of the infrastructure was linked with a computable general equilibrium (CGE) economics model that computes specific community resilience metrics in economic terms.  A population dislocation model provides resilience metrics related to socio-demographics such as population dislocation as a function of income or race/ethnicity.

## Prerequisites

The following modules are necessary to run this notebook. To ensure dependencies are correct, install all modules through **conda**. 

| Module | Version | Notes |
| --- | --- | --- |
| pyIncore | =>1.3.0 | see: https://incore.ncsa.illinois.edu/doc/incore/install_pyincore.html |
| pyIncore_viz | =>1.5.0 | see: https://incore.ncsa.illinois.edu/doc/pyincore_viz/index.html |
| matplotlib | 3.1.2 | used for plotting results |
| contextily | 0.99.0 | used for context geo-tiles |
| descartes | 1.1.0 | used for visualization of geometric objects as matplotlib paths |

Economic Computable General Equilibrium (CGE) Model uses the ipopt solver. The ipopt solver is installed with pyIncore, so the environment should have the solver in it. If you have a trouble to run CGE model later, please see the instruction at CGE section in this document. 

- Building data: 5dbc8478b9219c06dd242c0d
- Electric power substation data: 5d92355bb9219c06ae7e386a
- Hazard data:   5d07cbeab9219c065b080930
- Building Mapping:       5d8942dfb9219c068ea795ea

In [ ]:
import pandas as pd
import os
import json
from pathlib import Path
import contextily as ctx
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import random

from pyincore import IncoreClient, DataService, HazardService, Dataset, FragilityService, MappingSet
from pyincore import HHRSOutputProcess

from pyincore.analyses.capitalshocks import CapitalShocks
from pyincore.analyses.buildingfunctionality import BuildingFunctionality
from pyincore.analyses.buildingdamage import BuildingDamage
from pyincore.analyses.epfdamage import EpfDamage
from pyincore.utils.dataprocessutil import DataProcessUtil
from pyincore.analyses.montecarlofailureprobability import MonteCarloFailureProbability 
from pyincore.analyses.joplincge import JoplinCGEModel
from pyincore.analyses.housingrecoverysequential import HousingRecoverySequential
from pyincore.analyses.socialvulnerability import SocialVulnerability
from pyincore.analyses.populationdislocation import PopulationDislocation, PopulationDislocationUtil
from pyincore.utils.dataprocessutil import DataProcessUtil as util

# 2. Building Damage Analysis

The following steps are applied for analysis
- 2.1 IN-CORE Client and Services
- 2.2 Import Building data
- 2.3 Specify Hazard and Fragility
- 2.4 Setup and Run Damage analysis
- 2.5 Plot Results
- 2.6 Monte Carlo Simulation (MCS)

## 2.1 IN-CORE Client and Services

In [ ]:
# Create IN-CORE client and connection to services
client = IncoreClient()

data_services = DataService(client)

## 2.2 Import Building data

In [ ]:
building_dataset_id = "5dbc8478b9219c06dd242c0d"  # Load the Joplin building dataset.
buildings = Dataset.from_data_service(building_dataset_id, data_services)

archetype_mapping = "5fca92781ab3d87c35db1e54"

## 2.3 Specify Hazard and Fragility

In [ ]:
hazard_type = "tornado"
hazard_id = "5d07cbeab9219c065b080930"  # The simulated EF-5 tornado shows geographical locations and the range
                                        # of wind speed of tornado hazard in Joplin.

fragility_service = FragilityService(client)
mapping_id = "5d8942dfb9219c068ea795ea" # Create a mapping to assign tornado fragilities to 19 building archetypes.
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))

## 2.4 Setup and Run Damage analysis 

In [ ]:
# Analysis setup
result_name = "Joplin_bldg_dmg_result"

bldg_dmg = BuildingDamage(client)
bldg_dmg.set_input_dataset("buildings", buildings)
bldg_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)
bldg_dmg.set_parameter("result_name", result_name)
bldg_dmg.set_parameter("hazard_type", hazard_type)
bldg_dmg.set_parameter("hazard_id", hazard_id)
bldg_dmg.set_parameter("num_cpu", 4)

# Set up retrofit strategy
retrofit_strategy = "6614a2851f87410ae38a69f1" # 100% Retrofit Method 1
#retrofit_strategy = "6614a2b3099327393f84baa5" # 100% Retrofit Method 2
#retrofit_strategy = "6614a2e21f87410ae38a6aa2" # 100% Retrofit Method 3
retrofit_strategy_plan = Dataset.from_data_service(retrofit_strategy, data_services)
bldg_dmg.set_input_dataset("retrofit_strategy", retrofit_strategy_plan)

# Run building damage
bldg_dmg.run_analysis()

## 2.5 Plot analysis results

In [ ]:
# Print out 5 records from the analysis results
bldg_dmg.get_output_dataset('ds_result').get_dataframe_from_csv().head()

In [ ]:
# Adding information from inputs into analysis results
bldg_dmg_df = bldg_dmg.get_output_dataset("ds_result").get_dataframe_from_csv()
bldg_gdf = buildings.get_dataframe_from_shapefile()

arch_mapping_ds = Dataset.from_data_service(archetype_mapping, data_services)
arch_mapping = arch_mapping_ds.get_dataframe_from_csv()

print(arch_mapping)

max_dmg_state_df = DataProcessUtil.get_max_damage_state(bldg_dmg_df)
print(max_dmg_state_df)

ret_json = DataProcessUtil.create_mapped_dmg_result(bldg_gdf, max_dmg_state_df, arch_mapping, "max_state", "archetype")
print(ret_json)

bldg_dmg_gdf = bldg_gdf[['guid', 'archetype','geometry']].merge(bldg_dmg_df, on='guid')
bldg_dmg_gdf.head()

## 2.6 Monte Carlo Simulation (MCS)

In [ ]:
#### Obtain the building damage results
building_dmg_result = bldg_dmg.get_output_dataset('ds_result')

#### Convert the building damage results to dataframe
df_bldg_dmg = building_dmg_result.get_dataframe_from_csv()

df_bldg_dmg.head()

In [ ]:
num_samples = 500 # Require 500 samples for convergence - Selected smaller samples for testing 
result_name = "Joplin_mc_failure_probability_buildings_dmg_result"

mc_bldg = MonteCarloFailureProbability(client)

mc_bldg.set_input_dataset("damage", building_dmg_result)
mc_bldg.set_parameter("num_cpu", 8)
mc_bldg.set_parameter("num_samples", num_samples)
mc_bldg.set_parameter("damage_interval_keys", ["DS_0", "DS_1", "DS_2", "DS_3"])
mc_bldg.set_parameter("failure_state_keys", ["DS_2", "DS_3"])

mc_bldg.set_parameter("result_name", result_name) 

In [ ]:
mc_bldg.run_analysis() 

# Obtain buildings failure probabilities
building_failure_probability = mc_bldg.get_output_dataset('failure_probability')  

df_bldg_fail = building_failure_probability.get_dataframe_from_csv()
df_bldg_fail.head()

In [ ]:
# Obtain buildings sample damage states
sample_damage_states = mc_bldg.get_output_dataset('sample_damage_states')  
df_bldg_ds = sample_damage_states.get_dataframe_from_csv()
df_bldg_ds.head()

In [ ]:
building_damage_mcs_samples = mc_bldg.get_output_dataset('sample_failure_state')  # get buildings failure states

bdmcs = building_damage_mcs_samples.get_dataframe_from_csv()
bdmcs.head()

In [ ]:
ret_json = DataProcessUtil.create_mapped_func_result(bldg_gdf, 
                                                     bdmcs, 
                                                     arch_mapping, 
                                                     arch_col="archetype")
ret_json

# 4. Electric Power Facility (EPF) Damage Analysis

The following steps are applied for analysis
- 4.1 Import Electric Power Substation data
- 4.2 Specify Electric Power Substation Fragility and Mapping
- 4.3 Setup and Run Electric Power Substation Damage analysis
- 4.4 Plot Results for Electric Power Substation
- 4.5 Monte Carlo Simulation (MCS) of Electric Power Substation
- 4.6 Import Electric Power Generation Plants data
- 4.7 Specify Electric Power Generation Plants Fragility and Mapping
- 4.8 Setup and Run Electric Power Generation Plants Damage analysis
- 4.9 Plot Results for Electric Power Generation Plants
- 4.10 Monte Carlo Simulation (MCS) of Electric Power Generation Plants

## 4.1 Import Electric Power Substation data

In [ ]:
local_epf_ss = Dataset.from_data_service("5d92355bb9219c06ae7e386a", data_services)

## 4.2 Specify Electric Power Substation Fragility and Mapping

In [ ]:
# Fragility mapping
mapping_id = "5d8a326fb9219c068ea798e7"

## 4.3 Setup and Run Electric Power Substation Damage analysis

In [ ]:
# Run epf damage 
eq_epf_ss_dmg = EpfDamage(client)
eq_epf_ss_dmg.set_input_dataset("epfs", local_epf_ss)

# Load fragility mapping
fragility_service = FragilityService(client)
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
eq_epf_ss_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)

In [ ]:
result_name = "Joplin_epf_substations_dmg_result"
eq_epf_ss_dmg.set_parameter("result_name", result_name)
eq_epf_ss_dmg.set_parameter("hazard_type", hazard_type)
eq_epf_ss_dmg.set_parameter("hazard_id", hazard_id)
eq_epf_ss_dmg.set_parameter("num_cpu", 4)
eq_epf_ss_dmg.set_parameter("fragility_key", "substations")  # Define the result name, etc., and choose the number
                                                           # of CPU locally to run the model parallelly.

# Run Analysis
eq_epf_ss_dmg.run_analysis()

## 4.4 Plot Results for Electric Power Substation

In [ ]:
epf_ss_dmg_df=eq_epf_ss_dmg.get_output_dataset('result').get_dataframe_from_csv()
epf_ss_dmg_df.head()

In [ ]:
DSlist = np.array(['DS_0', 'DS_1', 'DS_2', 'DS_3', 'DS_4'])
spf_ss_likelyDS = DSlist[epf_ss_dmg_df.loc[:,['DS_0', 'DS_1', 'DS_2', 'DS_3','DS_4']].values.argmax(axis=1)]

epf_ss_dmg_df['likelyDS'] = spf_ss_likelyDS
keys, counts = np.unique(spf_ss_likelyDS, return_counts=True)

fig, ax = plt.subplots()
ax.bar(keys, counts)
ax.set_title("Damage State Distribution", fontsize=12)
ax.set_xlabel("Likely Damage State", fontsize=12)
ax.set_ylabel("Counts", fontsize=12)

In [ ]:
# Adding information from inputs into analysis results
epf_ss_gdf = local_epf_ss.get_dataframe_from_shapefile()
epf_ss_dmg_df = epf_ss_gdf[['guid', 'utilfcltyc','geometry']].merge(epf_ss_dmg_df, on='guid')
epf_ss_dmg_df.head()

In [ ]:
# Statistic summary for DS_4 state
epf_ss_dmg_df["DS_4"].describe()

In [ ]:
# Histogram plot for DS_4
ax = epf_ss_dmg_df['DS_4'].hist(bins=20, figsize=[10,5])
ax.set_title("complete damage distribution", fontsize=12)
ax.set_xlabel("complete damage value", fontsize=12)
ax.set_ylabel("counts", fontsize=12)

## 4.5 Monte Carlo Simulation (MCS) of Electric Power Substation

In [ ]:
num_samples = 500
mc_sub = MonteCarloFailureProbability(client)

result_name = "Joplin_EPN_substations_mc_failure_probability"
substation_dmg_result=eq_epf_ss_dmg.get_output_dataset('result')

mc_sub.set_input_dataset("damage", substation_dmg_result)
mc_sub.set_parameter("num_cpu", 8)
mc_sub.set_parameter("num_samples", num_samples)
mc_sub.set_parameter("damage_interval_keys", ["DS_0", "DS_1", "DS_2", "DS_3", "DS_4"])
mc_sub.set_parameter("failure_state_keys", ["DS_2", "DS_3", "DS_4"])

mc_sub.set_parameter("result_name", result_name) # name of csv file with results
mc_sub.run_analysis()

In [ ]:
substation_failure_probability = mc_sub.get_output_dataset('failure_probability')  # get buildings failure probabilities

df_substation_fail = substation_failure_probability.get_dataframe_from_csv()
df_substation_fail.head()

In [ ]:
substation_damage_mcs_samples = mc_sub.get_output_dataset('sample_failure_state')

sdmcs = substation_damage_mcs_samples.get_dataframe_from_csv()
sdmcs.head()

# 5. Physical Functionality Analysis

# 5.1. Functionality of Buildings and Substations

The MC samples from earlier subsection are used in this subsection to perform functionality analysis and determine the functionality state of each building. According to Almufti and Willford (2013), functionality is defined as the capacity of a component to serve its intended objectives consist of structural integrity, safety, and utilities (e.g., water and electricity). An individual building can be narrowly classified into five discrete states consist of the restricted entry (State 1), restricted use (State 2), preoccupancy (State 3), baseline functionality (State 4) and full functionality (State 5). In a broader classification, functionality states can be categorized into nonfunctional (States 1 to 3) and functional (States 4 and 5). This notebook relies on the latter broader classification approach to estimate the functionality state of each building and subsequently use functionality estimates to perfom functionality analysis by accounting for interdependency between buildings and lifeline networks (i.e., EPN) 

In [ ]:
depend_dict_id = "5dcf4a34b9219ca5e4118312"

bldg_func = BuildingFunctionality(client)

# Load the datasets of MC simulations for buildings
bldg_func.set_input_dataset("building_damage_mcs_samples", building_damage_mcs_samples)
bldg_func.set_input_dataset("substations_damage_mcs_samples", substation_damage_mcs_samples)
bldg_func.load_remote_input_dataset("interdependency_dictionary", depend_dict_id)

result_name = "Joplin_mcs_functionality_probability"
bldg_func.set_parameter("result_name", result_name)

In [ ]:
### Run the BuildingFunctionality module to obtain the building functionality probabilities
bldg_func.run_analysis() 

bldg_func_samples = bldg_func.get_output_dataset('functionality_samples')
df_bldg_samples = bldg_func_samples.get_dataframe_from_csv()

### Path to save the CSV file
#save_path = "functionality_samples.csv"
### Save the DataFrame to a CSV file
#df_bldg_samples.to_csv(save_path, index=True)

df_bldg_samples.head()

In [ ]:
dataframe=bldg_func.get_output_dataset('functionality_probability').get_dataframe_from_csv()

### Path to save the CSV file
#save_path = "functionality_probability.csv"
### Save the DataFrame to a CSV file
#dataframe.to_csv(save_path, index=True)

dataframe.head()

In [ ]:
ret_json = DataProcessUtil.create_mapped_func_result(bldg_gdf, 
                                                     df_bldg_samples, 
                                                     arch_mapping, 
                                                     arch_col="archetype")
ret_json

# 6. Sociological Analysis

This section introduces the sociology-based analysis which may be integrated with hazard and building damage analysis. In this example the damage to buildings drives the population dislocation algorithm at the household level.

# 6.1. Population Dislocation Model

Population dislocation refers to households that will be forced to leave their pre-event residence due to hazard related damages. Population dislocation is a function of structure value loss due to damage, neighborhood characteristics and structure type.

The population dislocation model depends first on the allocation of detailed housing unit and household characteristic data to each residential building within the community. The allocation of detailed household characteristic data provides an estimate of the number of people that live within each structure.

The model predicts which households within the community will be dislocated from their homes immediately following the hazard event.

## Use new pyincore-data utility to obtain Block Group Data for County

In [ ]:
bg_data = "5d4c9545b9219c0689b2358a"

In [ ]:
housing_unit_alloc = "5dc1c196b9219c06dd2e3f0b"
value_loss = "60354810e379f22e16560dbd"

In [ ]:
# Create Population dislocation analysis
pop_dis = PopulationDislocation(client)

In [ ]:
# Load analysis input datasets
pop_dis.load_remote_input_dataset("housing_unit_allocation", housing_unit_alloc)
pop_dis.load_remote_input_dataset("block_group_data", bg_data)
pop_dis.load_remote_input_dataset("value_loss_param", value_loss)
building_dmg_result = bldg_dmg.get_output_dataset('ds_result')
pop_dis.set_input_dataset("building_dmg", building_dmg_result)

In [ ]:
result_name = "pop-dislocation-results"
seed = 1111

pop_dis.set_parameter("result_name", result_name)
pop_dis.set_parameter("seed", seed)

In [ ]:
pop_dis.run_analysis()

# Retrieve result dataset
result = pop_dis.get_output_dataset("result")

# Convert dataset to Pandas DataFrame
df = result.get_dataframe_from_csv(low_memory=False)
df.head()

# 7.2. Sociological Resilience Metrics

## 7.2.1. Percent of Population Dislocation

To determine if a household dislocate a random value was uniformly sampled between 0 and 1: if this value was lower than the probability of dislocation, then the household was set to dislocate. The results below provide one example of how the output can be explored by population subgroups. The model is designed to run as part of a larger Monte Carlo Simulation.

In [ ]:
pop_result = pop_dis.get_output_dataset("result")

df = pop_result.get_dataframe_from_csv(low_memory=False)
# Select Housing units allocated to buildings

In [ ]:
table_title = "Total population by tenure status and dislocation."
pd.crosstab(df['ownershp'], df['dislocated'], df['numprec'], aggfunc = sum,
            dropna=True, margins=True, margins_name="Total").style.set_caption(table_title)

The variable ownershp represents the tenure status of households. When ownershp = 1 the housing unit is owner-occupied. When the ownershp = 2 the housing unit is renter-occupied. The results above provide the percent of the population predicted to dislocate based on tenure status. When the variable dislocated = True the household is predicted to dislocate.

# Capital Stock Shock and CGE Analysis

In [ ]:
# Joplin building to sector mapping table
building_to_sectors_id = "5f202d674620b643d787a5e7"
# Create Capital shocks analysis
capital_shocks = CapitalShocks(client)

# Load remote datasets
capital_shocks.load_remote_input_dataset("buildings_to_sectors", building_to_sectors_id)
# Set datasets
# Joplin building inventory
capital_shocks.load_remote_input_dataset("buildings", building_dataset_id)
# Joplin building failure probability
capital_shocks.set_input_dataset("failure_probability", building_failure_probability)

capital_shocks.set_parameter("result_name", "sector_shocks") # name of csv file with results

In [ ]:
# Run capital shocks analysis
capital_shocks.run_analysis()

sector_shocks_result = capital_shocks.get_output_dataset("sector_shocks")
sector_shocks_result.get_dataframe_from_csv()

In [ ]:
# Create Joplin CGE Model - TODO replace this with production client later
joplin_cge = JoplinCGEModel(client)

In [ ]:
# SAM
# Social accounting matrix (SAM) contains data for firms, households and government which are organized 
# in a way to represent the interactions of all three entities in a typical economy
SAM = "5dd85ae7b9219c06d4da8de4"

# CAPITAL COMP
# BB is a matrix which describes how investment in physical infrastructure is transformed into functioning capital such as commercial and residential buildings. 
# These data are collected from the Bureau of Economic Analysis (BEA).
BB = "5dc1e620b9219c06dd2f473a"

# MISC TABLES
IOUT = "5dc1e6d8b9219c06dd2f475e"  # This is matrix that describes the transfer of tax revenue collected by the local government to help finance local government expenditures. 
MISC = "5dc1e736b9219c06dd2f4782"  # This is the name of a file that contains data for commercial sector employment and physical capital. It also contains data for the number of households and working households in the economy. 
MISCH = "5dc1e7b5b9219c06dd2f47a6"  # A file that contains elasticities for the supply of labor with respect to paying income taxes.
LANDCAP = "5dc1e810b9219c06dd2f47ca"  # Contains information regarding elasticity values for the response of changes in the price of physical capital with respect to the supply of investment.
EMPLOY = "5dc1e85ab9219c06dd2f47ee"  # Table name containing data for commercial sector employment.
IGTD = "5dc1e895b9219c06dd2f4812"  # This variable represents a matrix describing the transfer of taxes collected to a variable which permits governments to spend the tax revenue on workers and intermediate inputs.
TAUFF = "5dc1e8eeb9219c06dd2f4836"  # Represents social security tax rates
JOBCR = "5dc1e962b9219c06dd2f487e"  # This is a matrix describing the supply of workers coming from each household group in the economy. 
OUTCR = "5dc1e9aeb9219c06dd2f48bc"  # This a matrix describing the number of workers who live in Joplin but commute outside of town to work.

In [ ]:
# Set analysis input datasets
joplin_cge.load_remote_input_dataset("SAM", SAM)
joplin_cge.load_remote_input_dataset("BB", BB)
joplin_cge.load_remote_input_dataset("IOUT", IOUT)
joplin_cge.load_remote_input_dataset("MISC", MISC)
joplin_cge.load_remote_input_dataset("MISCH", MISCH)
joplin_cge.load_remote_input_dataset("LANDCAP", LANDCAP)
joplin_cge.load_remote_input_dataset("EMPLOY", EMPLOY)
joplin_cge.load_remote_input_dataset("IGTD", IGTD)
joplin_cge.load_remote_input_dataset("TAUFF", TAUFF)
joplin_cge.load_remote_input_dataset("JOBCR", JOBCR)
joplin_cge.load_remote_input_dataset("OUTCR", OUTCR)
joplin_cge.set_input_dataset("sector_shocks", sector_shocks_result)

In [ ]:
# Set analysis parameters
joplin_cge.set_parameter("solver_path", "ipopt")
joplin_cge.set_parameter("model_iterations", 1)

In [ ]:
# Run CGE
joplin_cge.run_analysis()

In [ ]:
domestic_supply = joplin_cge.get_output_dataset('domestic-supply')
ds = domestic_supply.get_dataframe_from_csv()
# Display top 6 rows of output data
ds.head(6)

In [ ]:
gross_income = joplin_cge.get_output_dataset('gross-income')
gi = gross_income.get_dataframe_from_csv()
gi.head()

In [ ]:
pre_factor_demand = joplin_cge.get_output_dataset('pre-disaster-factor-demand')
pre_fd = pre_factor_demand.get_dataframe_from_csv()
pre_fd.head()

In [ ]:
household_count = joplin_cge.get_output_dataset('household-count')
hc = household_count.get_dataframe_from_csv()
hc.head()

In [ ]:
# Configure pyincore access and the analysis with the information above
social_vulnerability = SocialVulnerability(client)

social_vulnerability.set_parameter("result_name", "social_vulnerabilty")
social_vulnerability.load_remote_input_dataset("national_vulnerability_feature_averages", "63d178c2a011a9746c948115")
social_vulnerability.load_remote_input_dataset("social_vulnerability_demographic_factors", "63d17d89c4b5713b9c0bb8c1")

In [ ]:
# Run social vulnerability damage analysis
social_vulnerability.run_analysis()

In [ ]:
# Retrieve result dataset
sv_result = social_vulnerability.get_output_dataset("sv_result")

In [ ]:
housing_data = Dataset.from_data_service(housing_unit_alloc, data_services)
housing_inv = housing_data.get_dataframe_from_csv(low_memory=False)

In [ ]:
### Household Level Housing Recovery Analysis

In [ ]:
# Population dislocation result aggregated to the block group level.
# Transition probability matrix per social vulnerability level.
transition_probability_matrix = "60f5e2ae544e944c3cec0794"
# Initial mass probability function for stage 0.
initial_probability_vector = "60f5e918544e944c3cec668b"

# Create housing recovery instance
housing_recovery = HousingRecoverySequential(client)

# Load input datasets
housing_recovery.set_input_dataset("population_dislocation_block", pop_result)
housing_recovery.load_remote_input_dataset("tpm", transition_probability_matrix)
housing_recovery.load_remote_input_dataset("initial_stage_probabilities", initial_probability_vector)
housing_recovery.set_parameter("num_cpu", 4)

# Chain with SV output
housing_recovery.set_input_dataset('sv_result', sv_result)

# Initial value to seed the random number generator to ensure replication
seed = 1111
# A size of the analysis time step.
t_delta = 1.0
# Total duration.
t_final = 90.0


# Specify the result name
result_name = "housing_recovery_result"

# Set analysis parameters
housing_recovery.set_parameter("result_name", result_name)
housing_recovery.set_parameter("seed", seed)
housing_recovery.set_parameter("t_delta", t_delta)
housing_recovery.set_parameter("t_final", t_final)

housing_recovery.run()

In [ ]:
# Retrieve result dataset
result = housing_recovery.get_output_dataset("ds_result")

# Convert dataset to Pandas DataFrame
HH_stagehistory_DF = result.get_dataframe_from_csv(low_memory=False)

# Display top 5 rows of output data
HH_stagehistory_DF.head()

In [ ]:
Stage = HH_stagehistory_DF["1"]

In [ ]:
plt.rcParams["figure.figsize"] = [6, 4]

fig, ax = plt.subplots()
h_states = ['ES', 'TS', 'TH', 'PH', 'F']

counts = [(Stage==1.0).sum(),(Stage==2.0).sum(),(Stage==3.0).sum(),(Stage==4.0).sum(),(Stage==5.0).sum()]
bar_labels = ['1', '2', '3', '4','5']
bar_colors = ['tab:red', 'tab:orange', 'tab:green', 'tab:blue', 'tab:cyan']

ax.bar(h_states, counts, label=bar_labels, color=bar_colors)

ax.set_ylabel('Frequency')
ax.set_title('Housing Recovery Stage Summary')
ax.legend(title='Stage')

for bars in ax.containers:
    ax.bar_label(bars)
plt.show()
# ES: Emergency Shelter, TS: Temporary Shelter, TH: Temporary Housing, PH: Permanent Housing, F:Failure

In [ ]:
timesteps = ["1", "7", "13", "25", "49", "85"] # t0, t6, t12, t24, t48
print(HHRSOutputProcess.get_hhrs_stage_count(timesteps, HH_stagehistory_DF))

# 8. Retrofit Strategies
This section will walk you through the following steps to apply retrofit strategies to building damage analyses.
- 8.1 Building Damage Analysis for Non-Retrofittd Strategy
- 8.2 Building Damage Analysis for 3 Retrofit Strategies
- 8.3 Plot Damage State Distribution Across Analyses

## 8.1 Building Damage Analysis for Non-Retrofit Strategy
This step is the same as step 2-1 to step 2-4. We set up an analysis without any retrofit strategy.

In [ ]:
building_dataset_id = "5dbc8478b9219c06dd242c0d"  # Load the Joplin building dataset.
buildings = Dataset.from_data_service(building_dataset_id, data_services)

In [ ]:
archetype_mapping = "5fca92781ab3d87c35db1e54"
hazard_type = "tornado"
hazard_id = "5d07cbeab9219c065b080930"  # The simulated EF-5 tornado shows geographical locations and the range
                                        # of wind speed of tornado hazard in Joplin.

fragility_service = FragilityService(client)
mapping_id = "5d8942dfb9219c068ea795ea" # Create a mapping to assign tornado fragilities to 19 building archetypes.
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))

In [ ]:
# Analysis setup
result_name = "Joplin_bldg_dmg_result_w_retrofit"

bldg_dmg = BuildingDamage(client)
bldg_dmg.set_input_dataset("buildings", buildings)
bldg_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)
bldg_dmg.set_parameter("result_name", result_name)
bldg_dmg.set_parameter("hazard_type", hazard_type)
bldg_dmg.set_parameter("hazard_id", hazard_id)
bldg_dmg.set_parameter("num_cpu", 4)
#bldg_dmg.set_input_dataset("retrofit_strategy", retrofit_strategy_plan)

# Run building damage
bldg_dmg.run_analysis()

# Assuming you have a way to load your CSV into a DataF
bldg_dmg_df=bldg_dmg.get_output_dataset('ds_result').get_dataframe_from_csv()

## 8.2 Building Damage Analysis for 3 Retrofit Strategies
This section demonstrates how to apply three different retrofit strategies to building damage analyses. We utilise the same `BuildingDamage` object and run analyses with different `retrofit_strategy` parameters.

In [ ]:
# Retrofit Strategy A: 
retrofit_strategy_A_id = "6614a2851f87410ae38a69f1" # 100% Retrofit Method 1
retrofit_strategy_plan = Dataset.from_data_service(retrofit_strategy_A_id, data_services)
bldg_dmg.set_input_dataset("retrofit_strategy", retrofit_strategy_plan)
bldg_dmg.run_analysis()
bldg_dmg_df_A=bldg_dmg.get_output_dataset('ds_result').get_dataframe_from_csv()

# Retrofit Strategy B: 
retrofit_strategy_B_id = "6614a2b3099327393f84baa5" # 100% Retrofit Method 2
retrofit_strategy_plan = Dataset.from_data_service(retrofit_strategy_B_id, data_services)
bldg_dmg.set_input_dataset("retrofit_strategy", retrofit_strategy_plan)
bldg_dmg.run_analysis()
bldg_dmg_df_B=bldg_dmg.get_output_dataset('ds_result').get_dataframe_from_csv()

# Retrofit Strategy C: 
retrofit_strategy_C_id = "6614a2e21f87410ae38a6aa2" # 100% Retrofit Method 3
retrofit_strategy_plan = Dataset.from_data_service(retrofit_strategy_C_id, data_services)
bldg_dmg.set_input_dataset("retrofit_strategy", retrofit_strategy_plan)
bldg_dmg.run_analysis()
bldg_dmg_df_C=bldg_dmg.get_output_dataset('ds_result').get_dataframe_from_csv()

## 8.3 Plot Damage State Distribution Across Analyses

In [ ]:
# Create a single figure and a 1x3 grid of subplots
fig, axs = plt.subplots(1, 3, figsize=(10, 5)) # Adjust the figure size as needed
fig.suptitle('Damage State Distribution Across Analyses', fontsize=14)

# Flatten the axs array for easy iteration
axs = axs.flatten()

# List of DataFrames for iteration
df_list = [bldg_dmg_df_A, bldg_dmg_df_B, bldg_dmg_df_C]
# Titles for each subplot
titles = ['100% Retrofit Method 1', '100% Retrofit Method 2', '100% Retrofit Method 3'] 

# Loop through the DataFrames and their corresponding axes and titles
for ax, df, title in zip(axs, df_list, titles):
    # Perform your analysis here
    DSlist = np.array(['DS_0', 'DS_1', 'DS_2', 'DS_3'])
    spf_ss_likelyDS = DSlist[df.loc[:,['DS_0', 'DS_1', 'DS_2', 'DS_3']].values.argmax(axis=1)]
    df['likelyDS'] = spf_ss_likelyDS
    keys, counts = np.unique(spf_ss_likelyDS, return_counts=True)
    
    # Plot on the current axis
    ax.bar(keys, counts)
    ax.set_title(title, fontsize=12)
    ax.set_xlabel("Likely Damage State", fontsize=10)
    ax.set_ylabel("Counts", fontsize=10)
    ax.tick_params(axis='x', rotation=45) # Optional: Rotate x-axis labels for better readability
    
    # Ensure correct value annotations on each bar
    for bars in ax.containers:
        ax.bar_label(bars)

# Adjust layout for better spacing between subplots
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

# Show the plot
plt.show()